# dlt & SQLMesh Runner

## Install Dependencies

In [ ]:
%pip install "dlt[az,deltalake,filesystem,parquet]>=1.12.1"
%pip install "enlighten>=1.14.1"
%pip install "ibis==3.3.0"
%pip install "networkx>=3.5"
%pip install "sqlmesh[fabric,mssql] @ git+https://github.com/TobikoData/sqlmesh.git"

## Set Environment Vars

### Load From KeyVault

In [ ]:
import os

keyvault = "mattiasthalen-fabric"

credentials = [
    "CREDENTIALS__AZURE_TENANT_ID",
    "CREDENTIALS__AZURE_CLIENT_ID",
    "CREDENTIALS__AZURE_CLIENT_SECRET",
]

for credential in credentials:
    secret = credential.replace("_", "-")
    value = notebookutils.credentials.getSecret(f"https://{keyvault}.vault.azure.net/", secret)
    os.environ[credential] = value

    print(f"{credential} = {os.getenv(credential)}")

### Load From Variable Library

In [ ]:
variable_library = "project_vars"
project_vars = notebookutils.variableLibrary.getLibrary(variable_library)

vars = [
    "FABRIC__WORKSPACE_ID",
    "FABRIC__WAREHOUSE_ENDPOINT",
    "FABRIC__STATE_ENDPOINT",
    "FABRIC__STATE_DATABASE",
    "DESTINATION__BUCKET_URL",
]

for var in vars:
    value = project_vars[var]
    os.environ[var] = value
    print(f"{var} = {os.getenv(credential)}")


## Clone Project

In [ ]:
import git

organization = "mattiasthalen"
repo_name = "northwind-to-fabric"

url = f"https://github.com/{organization}/{repo_name}.git"

branch = "main"
code_path = f"/lakehouse/default/Files/{repo_name}"

try:
    repo = git.Repo.clone_from(
        url=url,
        to_path=code_path,
        branch=branch,
        depth=1,
        single_branch=True
    )
except git.GitCommandError as e:
    if e.status == 128:
        repo = git.Repo(code_path)
        origin = repo.remotes.origin
        origin.pull(branch)
    else:
        raise

## Run Project

### Run dlt

In [ ]:
import subprocess

cmd = ["python", "dlt/northwind.py", "prod"]
result = subprocess.run(
    cmd,
    cwd=code_path,
    text=True,
    check=True
) 

### Run SQLMesh

In [ ]:
import subprocess

cmd = ["sqlmesh", "plan", "prod", "--run", "--auto-apply", "--no-prompts"]
result = subprocess.run(
    cmd,
    cwd=code_path,
    text=True,
    check=True
) 